# 🤖 Churnito - Sistema de Predicción de Churn (Local)

**Sistema inteligente para predecir y prevenir la fuga de clientes usando Machine Learning y LLM**

**Versión optimizada para ejecución local** (con GPU si está disponible)

---

## 📋 Contenido:
1. Requisitos previos e instalación
2. Carga de datos
3. Entrenamiento del modelo de predicción
4. Descarga del LLM (Qwen2.5)
5. Sistema de chat interactivo con Churnito
6. Análisis y visualizaciones

---

## 1️⃣ Requisitos Previos

### Instalación de dependencias (ejecutar en terminal):

```bash
# Opción 1: Con pip
pip install torch torchvision torchaudio
pip install transformers>=4.30.0
pip install accelerate>=0.26.0
pip install datasets>=2.14.0
pip install scikit-learn>=1.3.0
pip install pandas numpy

# Opción 2: Con conda (recomendado)
conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia
conda install -c huggingface transformers
pip install accelerate datasets scikit-learn pandas numpy
```

### Requisitos de hardware:
- **CPU:** Funcional pero lento (~10-15 min entrenamiento)
- **GPU (recomendado):** NVIDIA con CUDA (~2-3 min entrenamiento)
- **RAM:** Mínimo 8GB, recomendado 16GB
- **Disco:** ~10GB para modelos y cache

## 2️⃣ Importar Bibliotecas y Verificar GPU

In [ ]:
import torch
import numpy as np
import pandas as pd
from pathlib import Path
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pickle
import os
import warnings
warnings.filterwarnings('ignore')

# Configuración
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# Verificar GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("✅ Bibliotecas importadas")
print(f"🖥️  PyTorch versión: {torch.__version__}")
print(f"🖥️  Dispositivo: {device}")
print(f"🖥️  CUDA disponible: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"🎮 GPU detectada: {torch.cuda.get_device_name(0)}")
    print(f"🎮 Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️  No se detectó GPU. El entrenamiento será en CPU (más lento)")

## 3️⃣ Cargar Dataset

El notebook buscará primero el archivo localmente, si no lo encuentra lo descargará.

In [ ]:
import urllib.request

# Buscar archivo local primero
filename = "Churn_Modelling.csv"
local_paths = [
    filename,
    f"../{filename}",
    f"data/{filename}"
]

dataset_path = None
for path in local_paths:
    if os.path.exists(path):
        dataset_path = path
        print(f"✅ Dataset encontrado en: {path}")
        break

# Si no existe, descargar desde GitHub
if dataset_path is None:
    url = "https://raw.githubusercontent.com/CuchoLeo/Fuga/main/Churn_Modelling.csv"
    print("📥 Descargando dataset desde GitHub...")
    urllib.request.urlretrieve(url, filename)
    dataset_path = filename
    print("✅ Dataset descargado")

# Cargar datos
df = pd.read_csv(dataset_path)

print(f"\n📊 Dataset cargado: {len(df)} registros, {len(df.columns)} columnas")
print(f"\nColumnas: {list(df.columns)}")
print(f"\n📈 Distribución de Churn:")
print(df['Exited'].value_counts())
print(f"\n🎯 Tasa de Churn: {df['Exited'].mean()*100:.2f}%")

# Vista previa
df.head()

## 4️⃣ Preprocesamiento de Datos

In [ ]:
print("🧹 Preprocesando datos...")

# Eliminar columnas no relevantes
cols_to_drop = ['RowNumber', 'CustomerId', 'Surname']
df_clean = df.drop([col for col in cols_to_drop if col in df.columns], axis=1)

# Codificar variables categóricas
label_encoders = {}
categorical_cols = ['Geography', 'Gender']

for col in categorical_cols:
    if col in df_clean.columns:
        le = LabelEncoder()
        df_clean[col] = le.fit_transform(df_clean[col])
        label_encoders[col] = le

# Separar features y target
X = df_clean.drop('Exited', axis=1)
y = df_clean['Exited']

# Normalizar features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

feature_names = X.columns.tolist()

# Dividir en train/test
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y.values, test_size=0.2, random_state=42, stratify=y
)

print(f"✅ Preprocesamiento completado")
print(f"📊 Train: {len(X_train)} muestras")
print(f"📊 Test: {len(X_test)} muestras")
print(f"📊 Features: {len(feature_names)}")

## 5️⃣ Crear Dataset para PyTorch

In [ ]:
class ChurnDataset(Dataset):
    """Dataset personalizado para predicción de churn"""
    
    def __init__(self, X, y, feature_names, tokenizer, max_length=256):
        self.X = X
        self.y = y
        self.feature_names = feature_names
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        # Crear descripción textual del cliente
        features = self.X[idx]
        text_parts = ["Cliente:"]
        
        for name, value in zip(self.feature_names, features):
            text_parts.append(f"{name}={value:.2f}")
        
        text = " ".join(text_parts)
        
        # Tokenizar
        encoding = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.y[idx], dtype=torch.long)
        }

print("✅ Clase ChurnDataset definida")

## 6️⃣ Entrenar Modelo de Predicción de Churn

Usamos DistilBERT para clasificación binaria (churn/no-churn).

⏱️ **Tiempo estimado:** 
- CPU: ~10-15 minutos
- GPU: ~2-3 minutos

In [ ]:
print("="*70)
print("🚀 ENTRENANDO MODELO DE PREDICCIÓN DE CHURN")
print("="*70)

# Cargar modelo base
model_name = "distilbert-base-uncased"
print(f"\n📦 Cargando {model_name}...")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    problem_type="single_label_classification"
)

print("✅ Modelo base cargado")

# Crear datasets
train_dataset = ChurnDataset(X_train, y_train, feature_names, tokenizer)
test_dataset = ChurnDataset(X_test, y_test, feature_names, tokenizer)

# ============================================================================
# CALCULAR CLASS WEIGHTS PARA MANEJAR DESBALANCE
# ============================================================================

from sklearn.utils.class_weight import compute_class_weight

classes = np.unique(y_train)
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y_train
)
class_weights = torch.tensor(class_weights_array, dtype=torch.float32)

print(f"\n⚖️  Balanceo de clases:")
print(f"Clase 0 (NO CHURN): {(y_train == 0).sum()} muestras, weight={class_weights[0]:.3f}")
print(f"Clase 1 (CHURN):    {(y_train == 1).sum()} muestras, weight={class_weights[1]:.3f}")
print(f"Ratio: {class_weights[1]/class_weights[0]:.2f}x más peso para clase minoritaria")

# ============================================================================
# CUSTOM TRAINER CON CLASS WEIGHTS (versión compatible con GPU/CPU)
# ============================================================================

class WeightedTrainer(Trainer):
    """Trainer personalizado que usa class weights en la función de pérdida"""

    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        # Extraer labels (sin modificar inputs)
        labels = inputs.get("labels")
        
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Calcular pérdida con class weights
        if self.class_weights is not None:
            # IMPORTANTE: Mover class_weights al mismo dispositivo que el modelo
            device = logits.device
            class_weights_device = self.class_weights.to(device)
            
            loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_device)
            loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        else:
            # Fallback a pérdida estándar
            loss = outputs.get("loss")

        return (loss, outputs) if return_outputs else loss

# Configuración de entrenamiento (optimizada para local)
training_args = TrainingArguments(
    output_dir="./churn_checkpoint",
    num_train_epochs=3,  # 3 épocas para mejor rendimiento
    per_device_train_batch_size=32 if torch.cuda.is_available() else 16,  # Ajustar según GPU
    per_device_eval_batch_size=32 if torch.cuda.is_available() else 16,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to="none",
    fp16=torch.cuda.is_available(),  # Activar mixed precision en GPU
)

# Función de métricas
def compute_metrics(eval_pred):
    from sklearn.metrics import confusion_matrix
    
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='binary', zero_division=0
    )
    
    # Matriz de confusión
    try:
        cm = confusion_matrix(labels, predictions)
        tn, fp, fn, tp = cm.ravel()
        
        print(f"\n📊 Matriz de Confusión:")
        print(f"   TN={tn}, FP={fp}")
        print(f"   FN={fn}, TP={tp}")
    except:
        pass
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Crear Trainer con class weights
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    class_weights=class_weights,  # Pasar class weights
)

# Entrenar
print("\n🏋️ Iniciando entrenamiento...")
print(f"🖥️  Dispositivo: {device}")
print(f"🖥️  Batch size: {training_args.per_device_train_batch_size}")
print(f"🖥️  Épocas: {training_args.num_train_epochs}")
print(f"🖥️  FP16 (mixed precision): {training_args.fp16}\n")

try:
    train_result = trainer.train()
    
    # Evaluar
    print("\n📊 Evaluando modelo...\n")
    eval_results = trainer.evaluate()

    print("\n" + "="*70)
    print("✅ ENTRENAMIENTO COMPLETADO")
    print("="*70)
    print(f"📊 Accuracy:  {eval_results['eval_accuracy']:.4f}")
    print(f"📊 Precision: {eval_results['eval_precision']:.4f}")
    print(f"📊 Recall:    {eval_results['eval_recall']:.4f}")
    print(f"📊 F1-Score:  {eval_results['eval_f1']:.4f}")

    # Interpretación
    print("\n💡 Interpretación:")
    print("   - Accuracy:  % de predicciones correctas (total)")
    print("   - Precision: De los que predecimos CHURN, % que realmente hacen churn")
    print("   - Recall:    De los que hacen CHURN, % que detectamos correctamente")
    print("   - F1-Score:  Balance entre Precision y Recall")

    # Advertencias y recomendaciones
    if eval_results['eval_precision'] < 0.5:
        print("\n⚠️  ADVERTENCIA: Precision baja. El modelo predice muchos falsos positivos.")
    if eval_results['eval_recall'] < 0.5:
        print("\n⚠️  ADVERTENCIA: Recall bajo. El modelo no detecta suficientes churners.")
    if eval_results['eval_f1'] > 0.7:
        print("\n✅ F1-Score bueno (>0.7). Modelo balanceado entre precision y recall.")

    # Guardar modelo
    model.save_pretrained("./churn_model")
    tokenizer.save_pretrained("./churn_model")

    # Guardar artefactos de preprocesamiento
    with open("./churn_model/preprocessing_artifacts.pkl", "wb") as f:
        pickle.dump({
            'scaler': scaler,
            'label_encoders': label_encoders,
            'feature_names': feature_names
        }, f)

    print("\n💾 Modelo guardado en: ./churn_model/")
    
except Exception as e:
    print(f"\n❌ Error durante entrenamiento: {e}")
    import traceback
    traceback.print_exc()

## 7️⃣ Descargar LLM para Chat (Qwen2.5)

⏱️ **Tiempo estimado:** 2-5 minutos (descarga ~3GB)

**Nota:** El LLM se cargará en GPU automáticamente si está disponible.

In [ ]:
print("🤖 Descargando LLM Qwen2.5-1.5B-Instruct...")
print("📥 Esto puede tardar 2-5 minutos (~3GB)\n")

llm_model_id = "Qwen/Qwen2.5-1.5B-Instruct"

llm_tokenizer = AutoTokenizer.from_pretrained(
    llm_model_id,
    trust_remote_code=True
)

# Cargar en GPU si está disponible
llm_model = AutoModelForCausalLM.from_pretrained(
    llm_model_id,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None,
    trust_remote_code=True
)

if not torch.cuda.is_available():
    llm_model = llm_model.to(device)

print("\n✅ LLM cargado exitosamente")
print(f"🖥️  Dispositivo LLM: {device}")
print("💬 Churnito está listo para conversar")

## 8️⃣ Sistema Churnito - Clase Principal

In [ ]:
class ChurnitoSystem:
    """Sistema completo de análisis de churn con chat conversacional"""
    
    def __init__(self, churn_model_path="./churn_model"):
        print("🔄 Inicializando Churnito...")
        
        # Cargar modelo de churn
        self.churn_tokenizer = AutoTokenizer.from_pretrained(churn_model_path)
        self.churn_model = AutoModelForSequenceClassification.from_pretrained(churn_model_path)
        self.churn_model.eval()
        
        # Mover a GPU si está disponible
        self.churn_model = self.churn_model.to(device)
        
        # Cargar artefactos
        with open(f"{churn_model_path}/preprocessing_artifacts.pkl", "rb") as f:
            artifacts = pickle.load(f)
            self.scaler = artifacts['scaler']
            self.label_encoders = artifacts['label_encoders']
            self.feature_names = artifacts['feature_names']
        
        # LLM (ya cargado globalmente)
        self.llm_tokenizer = llm_tokenizer
        self.llm_model = llm_model
        
        # Base de datos de clientes
        self.customer_database = df
        
        print(f"✅ Churnito inicializado en {device}")
    
    def predict_single_customer(self, customer_data):
        """Predice churn para un cliente"""
        # Codificar categóricas
        processed = customer_data.copy()
        for col_name, encoder in self.label_encoders.items():
            if col_name in processed:
                try:
                    processed[col_name] = encoder.transform([processed[col_name]])[0]
                except:
                    processed[col_name] = 0
        
        # Preparar features
        features = [float(processed.get(f, 0)) for f in self.feature_names]
        features_scaled = self.scaler.transform([features])
        
        # Crear texto
        text = "Cliente: " + " ".join([f"{n}={v:.2f}" for n, v in zip(self.feature_names, features_scaled[0])])
        
        # Predecir
        inputs = self.churn_tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=256)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = self.churn_model(**inputs)
            probabilities = torch.softmax(outputs.logits, dim=1)
            churn_prob = probabilities[0][1].item()
        
        return churn_prob
    
    def get_at_risk_customers(self, limit=10):
        """Obtiene clientes en riesgo"""
        df_sample = self.customer_database.sample(n=min(100, len(self.customer_database)), random_state=42)
        
        at_risk = []
        for idx, row in df_sample.iterrows():
            customer_data = {
                'CreditScore': row.get('CreditScore', 0),
                'Geography': row.get('Geography', ''),
                'Gender': row.get('Gender', ''),
                'Age': row.get('Age', 0),
                'Tenure': row.get('Tenure', 0),
                'Balance': row.get('Balance', 0),
                'NumOfProducts': row.get('NumOfProducts', 0),
                'HasCrCard': row.get('HasCrCard', 0),
                'IsActiveMember': row.get('IsActiveMember', 0),
                'EstimatedSalary': row.get('EstimatedSalary', 0)
            }
            
            churn_prob = self.predict_single_customer(customer_data)
            
            if churn_prob > 0.5:
                at_risk.append({
                    'customer_id': int(row.get('CustomerId', idx)),
                    'churn_probability': churn_prob,
                    'balance': customer_data['Balance'],
                    'age': customer_data['Age'],
                    'is_active': bool(customer_data['IsActiveMember'])
                })
        
        at_risk.sort(key=lambda x: x['churn_probability'], reverse=True)
        return at_risk[:limit]
    
    def get_statistics(self):
        """Obtiene estadísticas del dataset"""
        return {
            'total_customers': len(self.customer_database),
            'churned_customers': int(self.customer_database['Exited'].sum()),
            'churn_rate': float(self.customer_database['Exited'].mean()),
            'avg_balance': float(self.customer_database['Balance'].mean()),
            'avg_age': float(self.customer_database['Age'].mean())
        }
    
    def generate_structured_response(self, query, context):
        """Genera respuesta estructurada con datos reales"""
        response = []
        
        if "at_risk_customers" in context:
            at_risk = context["at_risk_customers"]
            if at_risk:
                high_value_count = sum(1 for c in at_risk if c['balance'] > 100000)
                inactive_count = sum(1 for c in at_risk if not c['is_active'])
                
                response.append(f"🎯 **Análisis de Clientes en Riesgo:**")
                response.append(f"   • {len(at_risk)} clientes identificados con alta probabilidad de churn")
                response.append(f"   • {high_value_count} son clientes de alto valor (Balance > $100k)")
                response.append(f"   • {inactive_count} clientes están inactivos")
                
                response.append("\n📊 **Top 5 Clientes Prioritarios:**")
                for i, customer in enumerate(at_risk[:5], 1):
                    prob_pct = customer['churn_probability'] * 100
                    response.append(
                        f"   {i}. Cliente #{customer['customer_id']}: {prob_pct:.1f}% riesgo, "
                        f"${customer['balance']:,.0f} balance\n"
                        f"      → {'🔴 INACTIVO' if not customer['is_active'] else '🟡 Activo'}"
                    )
        
        elif "statistics" in context:
            stats = context["statistics"]
            churn_rate = stats['churn_rate'] * 100
            
            response.append("📊 **Estadísticas Actuales:**")
            response.append(f"   • Total de clientes: {stats['total_customers']:,}")
            response.append(f"   • Tasa de churn: {churn_rate:.2f}%")
            response.append(f"   • Balance promedio: ${stats['avg_balance']:,.2f}")
            response.append(f"   • Edad promedio: {stats['avg_age']:.1f} años")
        
        return "\n".join(response) if response else "Churnito a tu servicio. ¿En qué puedo ayudarte?"
    
    def chat(self, query):
        """Procesa una consulta de chat"""
        query_lower = query.lower()
        context = {}
        
        # Detectar intenciones
        if any(word in query_lower for word in ["riesgo", "top", "clientes", "fuga", "muestra"]):
            context["at_risk_customers"] = self.get_at_risk_customers(limit=10)
        
        if any(word in query_lower for word in ["estadística", "tasa", "cuántos", "total"]):
            context["statistics"] = self.get_statistics()
        
        # Generar respuesta
        return self.generate_structured_response(query, context)

# Inicializar Churnito
churnito = ChurnitoSystem()
print("\n🤖 ¡Churnito está listo para conversar!")

## 9️⃣ Chat Interactivo con Churnito

¡Haz preguntas a Churnito sobre tus clientes!

In [ ]:
def chat_with_churnito():
    """Interfaz de chat simple"""
    print("="*70)
    print("💬 CHAT CON CHURNITO")
    print("="*70)
    print("\nEjemplos de preguntas:")
    print("  • Muéstrame los 10 clientes con mayor riesgo de fuga")
    print("  • ¿Cuál es la tasa de churn actual?")
    print("  • Dame estadísticas generales")
    print("\nEscribe 'salir' para terminar\n")
    print("="*70)
    
    while True:
        query = input("\n🧑 Tú: ")
        
        if query.lower() in ['salir', 'exit', 'quit']:
            print("\n👋 ¡Hasta luego!")
            break
        
        if not query.strip():
            continue
        
        response = churnito.chat(query)
        print(f"\n🤖 Churnito:\n{response}")
        print("\n" + "-"*70)

# Iniciar chat
chat_with_churnito()

## 🔟 Consultas Rápidas (sin interfaz interactiva)

In [ ]:
# Ejemplo 1: Clientes en riesgo
print("📊 CONSULTA: Clientes en riesgo\n")
response = churnito.chat("Muéstrame los 10 clientes con mayor riesgo de fuga")
print(response)

print("\n" + "="*70 + "\n")

# Ejemplo 2: Estadísticas
print("📊 CONSULTA: Estadísticas generales\n")
response = churnito.chat("Dame las estadísticas de churn")
print(response)

## 1️⃣1️⃣ Predicción para Cliente Específico

In [ ]:
# Ejemplo de cliente
ejemplo_cliente = {
    'CreditScore': 650,
    'Geography': 'France',
    'Gender': 'Female',
    'Age': 42,
    'Tenure': 2,
    'Balance': 125000,
    'NumOfProducts': 1,
    'HasCrCard': 1,
    'IsActiveMember': 0,
    'EstimatedSalary': 75000
}

churn_prob = churnito.predict_single_customer(ejemplo_cliente)

print("🔍 PREDICCIÓN PARA CLIENTE ESPECÍFICO")
print("="*70)
print(f"\n📊 Datos del cliente:")
for key, value in ejemplo_cliente.items():
    print(f"   • {key}: {value}")

print(f"\n🎯 Probabilidad de churn: {churn_prob*100:.2f}%")
print(f"🚦 Nivel de riesgo: {'🔴 ALTO' if churn_prob > 0.7 else '🟡 MEDIO' if churn_prob > 0.5 else '🟢 BAJO'}")

if churn_prob > 0.7:
    print("\n⚠️  RECOMENDACIÓN: Contactar inmediatamente para retención")
elif churn_prob > 0.5:
    print("\n💡 RECOMENDACIÓN: Implementar programa de retención preventivo")
else:
    print("\n✅ RECOMENDACIÓN: Monitoreo rutinario")

## 1️⃣2️⃣ Exportar Modelos (Opcional)

Comprime el modelo entrenado para compartir o respaldar.

In [ ]:
import shutil

# Comprimir modelo de churn
output_filename = "churn_model"
shutil.make_archive(output_filename, 'zip', './churn_model')

print(f"✅ Modelo comprimido: {output_filename}.zip")
print(f"📦 Tamaño: {os.path.getsize(f'{output_filename}.zip') / 1024**2:.2f} MB")
print(f"💾 Ubicación: {os.path.abspath(f'{output_filename}.zip')}")

---

## 🎉 ¡Felicidades!

Has completado la configuración de Churnito en tu entorno local.

### 💡 Ventajas de la versión local:
- ✅ Entrenamiento más rápido con GPU (si está disponible)
- ✅ 3 épocas para mejor rendimiento del modelo
- ✅ Mixed precision (FP16) para mayor velocidad
- ✅ Sin límites de tiempo de ejecución
- ✅ Modelos guardados localmente para reutilización

### 📚 Recursos adicionales:
- [Repositorio GitHub](https://github.com/CuchoLeo/Fuga)
- [Documentación completa](https://github.com/CuchoLeo/Fuga/blob/main/README.md)
- [Versión Colab](https://colab.research.google.com/github/CuchoLeo/Fuga/blob/main/Churnito_Colab.ipynb)

### 🤝 Creado por:
**Churnito Team** - Sistema de predicción de churn con IA

---